## Setup libraries

In [5]:
from math import sqrt
import numpy as np
import pandas as pd
from pandasql import sqldf
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping

# Setup tables

In [6]:
Moggio = read_csv("Moggiodata14_20.csv")
#Moggio = Moggio.loc[:, ["Date", "Ammonia", "PM10", "PM25",
#                        "Wind_speed", "Wind_direction", "Temperature", "Rainfall"]]

Moggio = Moggio.drop(['IDStation.1', 'NameStation.1', 'Date.1'], axis='columns')

print(Moggio.shape)
Moggio.head()



(2557, 25)


,Date,IDStation,NameStation,Ammonia,Arsenic,Benzo_a_pyrene,Cadmium,Lead,Nikel,NO,...,reg_Y_nn1_ID,IDStation.2,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Wind_speed_max,Wind_direction_max,Rainfall
0,2014-01-01T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,0.1,NaN,NaN,NaN,NaN,...,111,111,0.443554,99.0,-0.334965,45.545455,58.097902,1.805594,143.0,0.0
1,2014-01-02T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,111,111,0.355915,105.0,-0.486111,83.944444,0.000000,1.767361,153.0,7.8
2,2014-01-03T00:00:00Z,681,Moggio Loc Penscei,0.0,1.1,NaN,0.0,1.1,4.2,NaN,...,111,111,0.207631,120.0,2.220588,87.042254,30.514085,1.558451,152.0,0.0
3,2014-01-04T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,0.1,NaN,NaN,NaN,NaN,...,111,111,1.024926,175.0,2.137063,89.349650,0.202797,3.816783,164.0,35.8
4,2014-01-05T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,111,111,0.666721,94.0,2.416084,86.055944,10.951049,2.797203,136.0,25.0


In [8]:
#Primo quadrante
Q1 = Moggio.loc[((Moggio.Wind_direction > 0) &
                 (Moggio.Wind_direction < 90)), :]

a = Q1.shape[0]
arr1 = []
arr1 = [1 for i in range(a)]

del Q1['Wind_direction']
Q1["Wind_direction"] = arr1
columns_titles = ["index", "IDStation","NameStation","Date", "Ammonia", "PM10", "PM25",
                 "Wind_speed",  "Temperature","Relative_humidity","Global_radiation", "Rainfall", "Wind_direction" ]
                  
Q1 = Q1.reindex(columns=columns_titles)
Q1 = Q1.drop("index", axis = 1)

#Secondo quadrante
Q2 = Moggio.loc[((Moggio.Wind_direction > 90) &
                 (Moggio.Wind_direction < 180)), :]
a = Q2.shape[0]
arr2 = []
arr2 = [2 for i in range(a)]

del Q2['Wind_direction']
Q2["Wind_direction"] = arr2

#Terzo quadrante
Q3 = Moggio.loc[((Moggio.Wind_direction > 180) &
                 (Moggio.Wind_direction < 270)), :]

a = Q3.shape[0]
arr3 = []
arr3 = [3 for i in range(a)]

del Q3['Wind_direction']
Q3["Wind_direction"] = arr3

#Quarto quadrante
Q4 = Moggio.loc[((Moggio.Wind_direction > 270) &
                 (Moggio.Wind_direction < 360)), :]

a = Q4.shape[0]
arr4 = []
arr4 = [4 for i in range(a)]

del Q4['Wind_direction']
Q4["Wind_direction"] = arr4

Moggio = sqldf('''
SELECT *
FROM Q1
UNION
SELECT *
FROM Q2
UNION
SELECT *
FROM Q3
UNION
SELECT *
FROM Q4
''')
#del Moggio['Wind_direction']
Moggio.rename(columns={'Wind_direction': 'Quadrante'}, inplace=True)

Moggio

/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_3256/3377239663.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q1["Wind_direction"] = arr1
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_3256/3377239663.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2["Wind_direction"] = arr2
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_3256/3377239663.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

PandaSQLException: (sqlite3.OperationalError) SELECTs to the left and right of UNION do not have the same number of result columns
[SQL: 
SELECT *
FROM Q1
UNION
SELECT *
FROM Q2
UNION
SELECT *
FROM Q3
UNION
SELECT *
FROM Q4
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
Moggio

,Date,Ammonia,PM10,PM25,Wind_speed,Temperature,Rainfall,Quadrante
0,2014-01-01T00:00:00Z,0.0,6.0,7.0,0.443554,-0.334965,0.0,2
1,2014-01-02T00:00:00Z,0.0,6.0,5.0,0.355915,-0.486111,7.8,2
2,2014-01-03T00:00:00Z,0.0,11.0,4.0,0.207631,2.220588,0.0,2
3,2014-01-04T00:00:00Z,0.0,6.0,7.0,1.024926,2.137063,35.8,2
4,2014-01-05T00:00:00Z,0.0,3.0,8.0,0.666721,2.416084,25.0,2
...,...,...,...,...,...,...,...,...
2523,2020-12-27T00:00:00Z,0.0,4.0,5.0,0.314898,-3.706944,0.0,1
2524,2020-12-28T00:00:00Z,0.0,2.0,5.0,0.345959,-3.006250,27.6,2
2525,2020-12-29T00:00:00Z,0.0,2.0,0.0,1.031443,-0.051389,2.2,2
2526,2020-12-30T00:00:00Z,0.0,0.0,0.0,0.029681,-0.549306,0.0,3


In [ ]:
Moggio.to_csv('/Users/marcovinciguerra/Github/GitTesi/Python/Quadranti.csv')

